# Exploration 4

## 작사가 인공지능 만들기

## 평가

### 1. 가사 텍스트 생성 모델이 정상적으로 동작하는가? (텍스트 제너레이션 결과가 그럴듯한 문장으로 생성되는가?)
* 텍스트 생성 모델이 정상적으로 작동 함 (그럴듯한 문장이 생성됨)
* 예) smile => smile , i m not leaving you

### 2. 데이터의 전처리와 데이터셋 구성 과정이 체계적으로 진행되었는가? (특수문자 제거, 토크나이저 생성, 패딩처리 등의 과정이 빠짐없이 진행되었는가?)
* 강의 내용에 충실하게 전처리 과정을 모두 진행 함. 

### 3. 텍스트 생성모델이 안정적으로 학습되었는가? (텍스트 생성모델의 validation loss가 2.2 이하로 낮아졌는가?)
* 안정적으로 생성됨.   
* model.fit() 진행시 마지막 epoch 결과
  Epoch 10/10
  549/549 [==============================] - 92s 168ms/step - loss: 2.2998

## 회고1 (강의 내용과 프로젝트의 차이)
* 강의 내용과 프로젝트의 차이는 보다 많은 학습데이터
* 문장을 토큰화 할때 가사인 점을 감안하여 기존의 20개에서 15개로 변경 
  preprocess_sentence()함수에서 15개 보다 많은 토크의 생성을 막음.
* 훈련 데이터와 평가 데이터의 분리. sklearn 모듈의 train_test_split() 사용
  enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)
* 단어장 크기를 12000으로 변경

## 회고1 (강의 내용과 프로젝트의 차이)
* 훈련데이터와 테스트데이터를 나눈 후에 기존에 사용하던 변수를 찾는 과정에서 소스를 더욱더 꼼꼼하게 보게됨 (좋았다)
* 모델을 평가하는 것은 직접 단어를 입력하고, 결과를 받아 봐서 문장이 그럴사 한지 확인 해야 한다.
  단순히 이번 프로젝트이전에 evaluate 하는 것은 적합하지 않은 것 같음.
* 그래도 test 데이터를 이용하여 평가하려고 (테스트 데이터로 loss값을 찾으려함) 하여도 
  predict data를 구하는 방법을 고안해내지 못함
    지금의 프로그램은 predict 값이 단어 한번 사람이 직접 입력을 하고, 결과 문장을 받는 방법임.
    별도의 방법이 필요할 것 으로 생각됨 (사실 잘 모르는 부분입니다.)
* 결과적으로 loss값은 training 중에 나온 마지막 값으로 확인 함. 


#### 1. 시작 파일을  한 라인 단위로 읽어 온다. 


In [129]:
import glob
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 저장함
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


#### 2. 전처리 작업 (1)
   - 길이가 0인(내용이 없는) 문장은 제외 한다. 
   - 문장의 끝이 ":"인 문장(희곡에서 화자를 말하는 부분)은 제외한다.


In [130]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


In [ ]:
#### 3. 전처리작업 (2)
정규표현식(Regex)을 이용하여 필터링을 한다. 
3-1. 모든 문자를 소문자로 바꾸고, 양쪽 공백을 지웁니다  
3-2. 특수문자 양쪽에 공백을 넣고  
3-3. 여러개의 공백은 하나의 공백으로 바꿉니다  
3-4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다  
3-5. 다시 양쪽 공백을 지웁니다  
3-6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다


In [131]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 3-1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 3-2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3-3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 3-4
    sentence = sentence.strip() # 3-5
    
    # sentence의 단어 수가 13개를 넘으면 강재로 13까지로 잘라 줍니다.
    # 뒤에서 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 
    if len(sentence.split()) > 13:  
        sentence = ' '.join(sentence.split()[0:13])
        
    sentence = '<start> ' + sentence + ' <end>' # 3-6 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample sentence. sf df wefwef fwef wefw wefw wefwwef wef eg werf ergf ergf ergf ergf ergf ergf ergf"))


<start> this is sample sentence . sf df wefwef fwef wefw wefw wefwwef wef <end>


#### 3. 전처리작업 (3)
문장이 없거나, 화자를 나타낸 부분을 걸러낸다.

In [132]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]


['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

#### 3. 전처리작업 (4)
tokenize() 함수를 작성 한다. 
- 위에까지 전처리된 문자형식으로 정리된 데이터를 단어장인 tokenizer와 
  tokenizer를 이용한여 생성한 숫자로 이루어진 tensor 데이터를 생성 한다. 
- 입력 데이터의 시퀀스 길이를 일정하게 맞춰주기 위하여 padding 작업을 한다. 

In [133]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
def tokenize(corpus):
    # 12,000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12,000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   51    5 ...    0    0    0]
 [   2   17 2630 ...    0    0    0]
 [   2   32    7 ...   48    3    0]
 ...
 [   2  130    5 ...    6 1989    3]
 [   2   22   89 ...    6 1044    3]
 [   2    7   35 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f0f71609670>


In [134]:
# 단어장 내용 확인 
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


#### 3. 전처리작업 (5)
모델이 연속적인 학습으 하기 위하여 전처리를 또 한다. 

In [171]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])


[   2   51    5   92  293   65   56    9  952 6241    3    0    0    0]
[  51    5   92  293   65   56    9  952 6241    3    0    0    0    0]


#### 3. 전처리작업 (6)
dataset으로 만든 tensor를 train data와 test data로 분리합니다.

In [173]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

# print(len(enc_train))
# print(len(enc_val))
# print(len(enc_train) + len(enc_val))
print(len(enc_val) / (len(enc_train) + len(enc_val)) * 100)
print(len(enc_train) / (len(enc_train) + len(enc_val)) * 100)


20.000113798656038
79.99988620134395


#### 3. 전처리작업 (7)
데이터 전처리작업의 마직막으로 최종적으로 사용할 dataset을 만든다.

In [137]:
# BUFFER_SIZE = len(src_input)
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
# steps_per_epoch = len(src_input) // BATCH_SIZE
steps_per_epoch = len(enc_train) // BATCH_SIZE

# tokenizer가 구축한 단어사전 내 12,000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12,001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

#### 4. 모델 디자인
- layer는 다음과 같이 input layer 한개, output layer 한개, hidden layer 2개로 구성한다. 
  (embedding layer, LSTM hidden layer1, LSTM hidden layer2, dense layer)

In [138]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

# hiperparameter 조정 하자!
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)


In [184]:
model.summary()

Model: "text_generator_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  3072256   
_________________________________________________________________
lstm_4 (LSTM)                multiple                  5246976   
_________________________________________________________________
lstm_5 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense_2 (Dense)              multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [139]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~

for src_sample, tgt_sample in dataset.take(1): break 
                                                     # dataset.take(1)를 통해서 1개의 배치, 
                                                     # 즉 256개의 문장 데이터를 가져온 것
# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

# shape=(256, 20, 7001)
# 256 : BATCH_SIZE
# 20 : 데이터셋의 max_len
# 7001 : VOCAB_SIZE = tokenizer.num_words + 1  
# tokenizer.num_words : 7000

# shape=(256, 14, 12001)
# 256 : BATCH_SIZE
# 14 : 데이터셋의 max_len
# 12,001 : VOCAB_SIZE = tokenizer.num_words + 1  
# tokenizer.num_words : 12000

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-2.01259303e-04, -3.92656475e-05, -2.38218432e-04, ...,
         -1.83112948e-04,  1.38595889e-04,  6.69659712e-05],
        [-6.62942650e-04,  2.87055300e-04, -2.71077181e-04, ...,
         -3.18025443e-04, -2.18537643e-05,  2.36611988e-04],
        [-8.69255047e-04,  5.26356453e-04, -3.78989003e-04, ...,
         -5.52613172e-04, -1.73657856e-04,  1.61345481e-04],
        ...,
        [-5.37862477e-04,  3.70192080e-04, -6.68391469e-04, ...,
          5.06643031e-04,  7.77111505e-04,  9.65363870e-05],
        [-5.62284666e-04,  7.64305296e-05, -7.10894179e-04, ...,
          5.49925724e-04,  9.42286570e-04, -7.93044892e-05],
        [-5.69545547e-04, -2.85156217e-04, -7.54587585e-04, ...,
          6.01964653e-04,  1.05449499e-03, -3.90619156e-04]],

       [[-2.01259303e-04, -3.92656475e-05, -2.38218432e-04, ...,
         -1.83112948e-04,  1.38595889e-04,  6.69659712e-05],
        [-2.04585769e-04, -1.74702422e-04, -2

#### 5. 모델 학습
- optimizer와 loss를 구성하고 학습시킵니다 !!!!!!!! 

In [140]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
# 
# tf.test.is_gpu_available() 소스를 실행해 텐서플로우가 GPU를 잘 사용하고 있는지 확인가능 함.

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, 
    reduction='none'
)
# loss = tf.keras.losses.SparseCategoricalCrossentropy(
#     from_logits=True,
#     reduction='none'
# )

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
549/549 [==============================] - 95s 169ms/step - loss: 3.6755
Epoch 2/10
549/549 [==============================] - 93s 169ms/step - loss: 3.2013
Epoch 3/10
549/549 [==============================] - 93s 168ms/step - loss: 3.0154
Epoch 4/10
549/549 [==============================] - 92s 168ms/step - loss: 2.8794
Epoch 5/10
549/549 [==============================] - 92s 168ms/step - loss: 2.7611
Epoch 6/10
549/549 [==============================] - 92s 168ms/step - loss: 2.6558
Epoch 7/10
549/549 [==============================] - 92s 168ms/step - loss: 2.5574
Epoch 8/10
549/549 [==============================] - 93s 169ms/step - loss: 2.4665
Epoch 9/10
549/549 [==============================] - 93s 168ms/step - loss: 2.3808
Epoch 10/10
549/549 [==============================] - 92s 168ms/step - loss: 2.2998


#### 6. 모델 학습
- 작문을 시켜보고 직접 평가하는 함수 작성
- generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행합니다.

In [141]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=14):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

#### 7. 테스트 및 결과 확인 

In [176]:
# 테스트 시작 
generate_text(model, tokenizer, init_sentence="<start> smile")


'<start> smile , i m not leaving you <end> '